In [83]:
#Installing required libraries
!pip install pandas numpy matplotlib bokeh pathway

In [84]:
!pip install pathway bokeh --quiet

In [85]:
#Importing standard libraries
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
import pathway as pw  # for streaming simulation
from scipy.spatial import cKDTree
from bokeh.models import ColumnDataSource
import datetime
from datetime import datetime

In [86]:
#Loading the dataset
# Assuming the dataset is uploaded directly to the Colab environment
df = pd.read_csv('dataset.csv')

In [87]:
#Displaying the first rows and info
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
 12  Timestamp               18368 non-null  object 
 13  occupancy_rate          18368 non-null  float64
dtypes: float64(3), int64(5), object(6)
mem

,ID,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay,occupancy_rate
count,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000,18368.000000
mean,9183.500000,1605.214286,25.706547,90.751170,731.084059,4.587925,0.150915,0.509119
std,5302.529208,1131.153886,1.582749,3.536636,621.164982,2.580062,0.357975,0.246143
min,0.000000,387.000000,20.000035,78.000003,2.000000,0.000000,0.000000,0.003466
25%,4591.750000,577.000000,26.140048,91.727995,322.000000,2.000000,0.000000,0.307110
50%,9183.500000,1261.000000,26.147482,91.729511,568.000000,4.000000,0.000000,0.496124
75%,13775.250000,2803.000000,26.147541,91.736172,976.000000,6.000000,0.000000,0.701031
max,18367.000000,3883.000000,26.150504,91.740994,3499.000000,15.000000,1.000000,1.041344


In [88]:
#Data Cleaning & Preprocessing
if {'LastUpdatedDate', 'LastUpdatedTime'}.issubset(df.columns):
  df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
else:
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # fallback

# Sorting the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

if {'Occupancy', 'Capacity'}.issubset(df.columns):
  df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
#Check for nulls
null_counts= df.isnull().sum()
print("Null count by columns is:",null_counts)
#If there are missing rows dropping them
dir_to_drop= null_counts[null_counts>0].index.tolist()
if dir_to_drop:
    df.dropna(subset=dir_to_drop, inplace=True)
    print(f"Dropping rows with missing values in columns: {dir_to_drop}")
else:
    print("No rows with missing values found.")

Null count by columns is: ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
Timestamp                 0
occupancy_rate            0
dtype: int64
No rows with missing values found.


In [89]:
#Verifying no remaining nulls
null_counts= df.isnull().sum()
print("Null count by columns is:",null_counts)

Null count by columns is: ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
Timestamp                 0
occupancy_rate            0
dtype: int64


In [90]:
#Histogram for occupancy rate
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook
output_notebook()

# Calculating histogram values manually
hist, edges = np.histogram(df['occupancy_rate'], bins=30)

# Creating Bokeh plot
hist_source = ColumnDataSource(data=dict(
    top=hist,
    left=edges[:-1],
    right=edges[1:]
))

hist_fig = figure(
    title="Occupancy Rate Distribution",
    x_axis_label='Occupancy Rate',
    y_axis_label='Frequency',
    width=600,
    height=400
)

hist_fig.quad(top='top', bottom=0, left='left', right='right',
              source=hist_source, fill_color="navy", line_color="white", alpha=0.7)

show(hist_fig)


In [91]:
#Time series of average occupancy over time
# Ensuring timestamp column is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Resampling data hourly
ts = df.set_index('Timestamp').resample('h')['Occupancy'].mean().reset_index()

# Preparing Bokeh data source
ts_source = ColumnDataSource(ts)

# Creating Bokeh time series plot
ts_fig = figure(x_axis_type='datetime',
                title='Hourly Average Occupancy Over Time',
                x_axis_label='Time',
                y_axis_label='Average Occupancy',
                width=800, height=400)

ts_fig.line(x='Timestamp', y='Occupancy', source=ts_source, line_width=2, color='green')

show(ts_fig)

Model 1: Baseline Linear Model

In [92]:
#Implementing and visualizing the simple linear pricing rule
#Parameters
alpha= 0.5
base_price = 10
def baseline_pricing(prices, rates, alpha):
   for i in range(1, len(rates)):
        prices[i] = prices[i - 1] + alpha * rates[i - 1]
   return prices

In [93]:
# Choosing one lot and calculate occupancy rate over time
first_lot = df['SystemCodeNumber'].unique()[0]
lot_df = df[df['SystemCodeNumber'] == first_lot].copy()

# Resampling occupancy rate every 30 minutes, filling forward
ts = lot_df.set_index('Timestamp').resample('30min')['Occupancy'].mean().ffill()

# Aligning times and rates
times = ts.index.to_list()
rates = ts.values

# Generating price array
prices = np.full(len(rates), base_price)
prices = baseline_pricing(prices, rates, alpha)

In [94]:
#Plot
# Bokeh line chart for Baseline Price Evolution
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show

baseline_source = ColumnDataSource(data=dict(
    time=times,
    price=prices
))

baseline_fig = figure(
    x_axis_type='datetime',
    title='Baseline Price Evolution (Model 1)',
    x_axis_label='Time',
    y_axis_label='Price',
    width=800,
    height=400
)

baseline_fig.line('time', 'price', source=baseline_source, line_width=2, color='blue')

show(baseline_fig)

In [95]:
print(df.columns)

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'Timestamp',
       'occupancy_rate'],
      dtype='object')


Model 2: Demand-Based Price Function

In [96]:
#Feature normalization
import pandas as pd

# Identifying numerical and categorical features
numerical_features = ['Occupancy', 'QueueLength', 'IsSpecialDay']
categorical_features = ['VehicleType', 'TrafficConditionNearby']

# Applying one-hot encoding to categorical features
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Defining the list of features to scale (numerical + encoded categorical)
features_to_scale = numerical_features + [col for col in df_encoded.columns if any(col.startswith(f) for f in categorical_features)]

df_scaled = df_encoded.copy()
for col in features_to_scale:
    # Convert boolean column to int if needed
    if df_scaled[col].dtype == bool:
        df_scaled[col] = df_scaled[col].astype(int)

    min_val = df_scaled[col].min()
    max_val = df_scaled[col].max()

    if max_val != min_val:
        df_scaled[col] = (df_scaled[col] - min_val) / (max_val - min_val)
    else:
        df_scaled[col] = 0.0  # or keep it unchanged if no variation

# Displaying the first few rows of the scaled DataFrame and its info to verify
display(df_scaled.head())
display(df_scaled.info())

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,occupancy_rate,VehicleType_car,VehicleType_cycle,VehicleType_truck,TrafficConditionNearby_high,TrafficConditionNearby_low
0,0,BHMBCCMKT01,577,26.144536,91.736172,0.016872,0.066667,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.105719,1.0,0.0,0.0,0.0,1.0
1,7872,Broad Street,690,26.137958,91.740994,0.050329,0.133333,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.257971,1.0,0.0,0.0,0.0,1.0
2,5248,BHMNCPHST01,1200,26.140014,91.731000,0.067200,0.133333,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.197500,0.0,0.0,0.0,0.0,1.0
3,3936,BHMMBMMBX01,687,20.000035,78.000003,0.074921,0.133333,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.384279,1.0,0.0,0.0,0.0,1.0
4,17056,Shopping,1920,26.150504,91.733531,0.175007,0.133333,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.319792,0.0,1.0,0.0,0.0,1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           18368 non-null  int64         
 1   SystemCodeNumber             18368 non-null  object        
 2   Capacity                     18368 non-null  int64         
 3   Latitude                     18368 non-null  float64       
 4   Longitude                    18368 non-null  float64       
 5   Occupancy                    18368 non-null  float64       
 6   QueueLength                  18368 non-null  float64       
 7   IsSpecialDay                 18368 non-null  float64       
 8   LastUpdatedDate              18368 non-null  object        
 9   LastUpdatedTime              18368 non-null  object        
 10  Timestamp                    18368 non-null  datetime64[ns]
 11  occupancy_rate               18368 non-nu

None

In [97]:
#Computing a demand score (weighted sum)
weights = {
    'occupancy': 0.4,
    'queue_length': 0.3,
    'is_special_day': 0.1
}

for col in df_scaled.columns:
    if "TrafficConditionNearby" in col:
        weights[col] = 0.1

df_scaled['demand_score'] = sum(df_scaled[col] * weights[col] for col in weights if col in df_scaled.columns)
df_scaled['demand_price'] = base_price * (1 + df_scaled['demand_score'])
df_scaled['demand_price'] = df_scaled['demand_price'].clip(lower=0.5 * base_price, upper=2 * base_price)


# Ensuring all weighted columns exist in df_scaled
weighted_columns = [f for f in weights.keys() if f in df_scaled.columns]

df_scaled['demand_score'] = sum(df_scaled[f] * weights[f] for f in weighted_columns)

# Displaying the first few rows with the new demand score
display(df_scaled[['Occupancy', 'QueueLength', 'IsSpecialDay', 'demand_score']].head())

,Occupancy,QueueLength,IsSpecialDay,demand_score
0,0.016872,0.066667,0.0,0.1
1,0.050329,0.133333,0.0,0.1
2,0.067200,0.133333,0.0,0.1
3,0.074921,0.133333,0.0,0.1
4,0.175007,0.133333,0.0,0.1


In [98]:
#Mapping the demand score to price
lambda_param = 1.0
df_scaled['demand_price'] = base_price * (1 + lambda_param * df_scaled['demand_score'])
#Enforcing bounds of base price
df_scaled['demand_price'] = df_scaled['demand_price'].clip(lower=0.5 * base_price, upper=2 * base_price)

 Model 3: Competitive Pricing Model

In [99]:
#Identifying nearest lot(by latitude/longitude)
from scipy.spatial import cKDTree
#Building a tree of lot coordinates
groups = df_scaled[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates().set_index('SystemCodeNumber')
coords = groups[['Latitude', 'Longitude']].values
tree = cKDTree(coords)

In [100]:
_, idxs = tree.query(coords, k=2)
lots = groups.copy()
lots['nearest_lot'] = groups.index[idxs[:, 1]]

In [101]:
#Finding competitor's prices
lots['competitor_price'] = base_price * (1 + 0.1 * np.sin(np.arange(len(lots))))

In [102]:
#Adjusting the price based on it
df_comp = df_scaled.merge(lots[['competitor_price']], left_on='SystemCodeNumber', right_index=True)

In [103]:
#If competitor is cheap apply a small premium or else go with demand price
df_comp['final_price'] = np.where(
    df_comp['competitor_price'] < df_comp['demand_price'],
    df_comp['competitor_price'] * 1.05,
    df_comp['demand_price']
)
df_comp['reroute'] = (df_comp['Occupancy'] > 0.9) & (df_comp['competitor_price'] < df_comp['final_price'])
print(df_comp[df_comp['reroute']][['SystemCodeNumber', 'reroute']])

       SystemCodeNumber  reroute
89     Others-CCCPS135a     True
105    Others-CCCPS135a     True
114    Others-CCCPS135a     True
128    Others-CCCPS135a     True
151    Others-CCCPS135a     True
...                 ...      ...
17309  Others-CCCPS135a     True
18269  Others-CCCPS135a     True
18272  Others-CCCPS135a     True
18288  Others-CCCPS135a     True
18302  Others-CCCPS135a     True

[246 rows x 2 columns]


In [104]:
df_stream = df.copy()

# Making sure timestamp is datetime type
df_stream['Timestamp'] = pd.to_datetime(df_stream['Timestamp'])

# Saving as CSV for Pathway
df_stream.to_csv("dataset.csv", index=False, date_format="%Y-%m-%d %H:%M:%S")

In [105]:
df_stream = df.copy()

# Renaming columns
df_stream.rename(columns={
    "SystemCodeNumber": "lot_id",
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Capacity": "capacity",
    "Occupancy": "occupancy",
    "QueueLength": "queue_length",
    "VehicleType": "vehicle_type",
    "TrafficConditionNearby": "traffic_level",
    "IsSpecialDay": "is_special_day",
    "Timestamp": "timestamp"
}, inplace=True)

# Saving the CSV
df_stream[[
    "timestamp", "occupancy", "capacity", "lot_id", "latitude", "longitude",
    "queue_length", "vehicle_type", "traffic_level", "is_special_day"
]].to_csv("parking_stream.csv", index=False)

In [106]:
import pathway as pw
class ParkingInput(pw.Schema):
    timestamp:str
    lot_id: str
    latitude: float
    longitude: float
    capacity: int
    occupancy: int
    queue_length: int
    vehicle_type: str
    traffic_level: str
    is_special_day: int
stream_data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingInput, input_rate=1000)

In [107]:

# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = stream_data.with_columns(
    t = stream_data.timestamp.dt.strptime(fmt),
    day = stream_data.timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [108]:
@pw.udf
def calculate_price(capacity, occupancy, queue_length, traffic_level, is_special_day, vehicle_type):
    base_price = 10

    # Avoiding division by zero
    if capacity == 0:
        occupancy_rate = 0
    else:
        occupancy_rate = occupancy / capacity

    # Encoding vehicle type as weights
    if vehicle_type == "car":
        vehicle_weight = 0.05
    elif vehicle_type == "bike":
        vehicle_weight = 0.02
    elif vehicle_type == "truck":
        vehicle_weight = 0.08
    else:
        vehicle_weight = 0.03  # default fallback

    # Encoding traffic level manually (0 = low, 1 = medium, 2 = high)
    if traffic_level == 2:
        traffic_penalty = 0.15
    elif traffic_level == 1:
        traffic_penalty = 0.05
    else:
        traffic_penalty = 0.0

    # Demand function (based on your weights)
    demand = (
        0.4 * occupancy_rate +
        0.3 * queue_length +
        0.1 * is_special_day +
        vehicle_weight -
        traffic_penalty
    )

    # Demand-based price
    price = base_price * (1 + 1.0 * demand)

    # Clip price between 0.5x and 2x of base price
    price = max(0.5 * base_price, min(2 * base_price, price))
    return price

# Convert timestamp string to datetime within Pathway using strptime
stream_data_dt = stream_data.select(
    timestamp=stream_data.timestamp,
    lot_id=stream_data.lot_id,
    latitude=stream_data.latitude,
    longitude=stream_data.longitude,
    capacity=stream_data.capacity,
    occupancy=stream_data.occupancy,
    queue_length=stream_data.queue_length,
    vehicle_type=stream_data.vehicle_type,
    traffic_level=stream_data.traffic_level,
    is_special_day=stream_data.is_special_day
)


prices = stream_data_dt.select(
    lot_id=stream_data_dt.lot_id,
    timestamp=stream_data_dt.timestamp,
    price=calculate_price(
        stream_data_dt.capacity,
        stream_data_dt.occupancy,
        stream_data_dt.queue_length,
        stream_data_dt.traffic_level,
        stream_data_dt.is_special_day,
        stream_data_dt.vehicle_type
    )
)

In [114]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from pathway.io.python import ConnectorObserver
import pandas as pd

output_notebook()

# Creating Bokeh plot
source_bokeh = ColumnDataSource(data=dict(time=[], price=[]))
p = figure(x_axis_type='datetime', title='Real-Time Parking Price')
p.line('time', 'price', source=source_bokeh, line_width=2)

# Fix the observer class
class BokehObserver(pw.io.python.ConnectorObserver):
    def __init__(self):
        super().__init__()
    def on_change(self, key, row, time, is_addition):
        if row is None:
            return
        from pandas import to_datetime  # ensure import is available
        ts_obj = to_datetime(row['timestamp'])
        source_bokeh.stream({
            'time': [ts_obj],
            'price': [row['price']]
        }, rollover=200)

# Connecting Pathway table to observer
observer = BokehObserver()
pw.io.python.write(prices, observer)

# Showing plot and running Pathway
show(p)

In [115]:
pw.run()

Output()

TypeError: BokehObserver.on_change() got an unexpected keyword argument 'key'